In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
from IPython.display import Image,display, Audio
import dalmatian as dm
from IPython.core.display import HTML 
from src.CCLE_postp_function import *
import sys
sys.path.insert(0, '../JKBio/')
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
namespace1="broad-genomics-delivery"
workspace1="Getz_IBM_CellLines_Exomes"
namespace2="broad-firecloud-ccle"
workspace2="CCLE_DepMap_WES"
namespace3="broad-genomics-delivery"
workspace3="CCLE_DepMap_WES"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_WES_CN_hg38"
source1="ibm"
source2="ccle"
source3="ccle"
sample_set_id = "19Q4"
release = sample_set_id

In [ ]:
wm1 = dm.WorkspaceManager(namespace1, workspace1)
wm2 = dm.WorkspaceManager(namespace2, workspace2)
wm3 = dm.WorkspaceManager(namespace3, workspace3)
# wm4 = dm.WorkspaceManager(namespace4, workspace4)

refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [ ]:
# #####
# ####
# tmp = terra.changeGSlocation(refnamespace+'/'+refworkspace, newgs='gs://ccle_bams', onlycol = ['WES_bai','WES_bam'], entity = 'samples', dry_run = True)

In [ ]:
# sample_info = pd.read_csv('../workspace_safety/DepMap_Mutation_Calling_CGA_pipeline/edited/sample.tsv', sep = "\t",na_filter= False)
# sample_info = sample_info.rename(columns={"entity:sample_id":'sample_id'})
# sample_info = sample_info.set_index('sample_id')
# sampleset_members = pd.read_csv('../workspace_safety/DepMap_Mutation_Calling_CGA_pipeline/edited/sample_set_membership.tsv', sep = "\t",na_filter= False)
# sampleset_members = sampleset_members.rename(columns={"membership:sample_set_id":'sample_set_id'})
# sampleset_members = sampleset_members.set_index('sample_set_id')

# sampleset_entity = pd.read_csv('../workspace_safety/DepMap_Mutation_Calling_CGA_pipeline/sample_set/sample_set_entity.tsv', sep = "\t", na_filter= False)
# sampleset_entity = sampleset_entity.set_index('entity:sample_set_id')


## Uploading sample tsv to Terra from local file

In [ ]:
sample_info = pd.read_csv('../workspace_safety/DepMap_WES_CN_hg38/v4/sample.tsv', sep = "\t", na_filter= False)
print(sample_info.shape)
new_sample_info = pd.read_csv('../workspace_safety/DepMap_WES_CN_hg38/19Q4_extra_new_cn_samples.tsv', sep = "\t", na_filter= False)
new_sample_info = new_sample_info[0:4] # check: IDK why it had extra empty rows
sample_info = pd.concat([sample_info, new_sample_info], axis=0, sort=True)
sample_info = sample_info.rename(columns={"entity:sample_id":'sample_id'})
sample_info.drop(['readgroup_ubams'], axis=1, inplace=True)
sample_info = sample_info.set_index('sample_id')
sample_info.index.name = 'entity:sample_id' ## check: maybe delete this
sample_info.drop(['Unnamed: 6'], axis = 1, inplace=True)
print(sample_info.shape)

In [ ]:
# Run this chunk to save the sample_info TSV to a file. I highly recommend this when running a pipeline on a new batch.
# This way, if anything goes wrong in the workspace, you can fall back to this.
filepath = '../workspace_safety/DepMap_WES_CN_hg38/v4/%s_sample_info.tsv' % 'edited_'
sample_info.to_csv(filepath, sep='\t')

In [ ]:
# read in the file you just saved
filepath = '../workspace_safety/DepMap_WES_CN_hg38/v4/%s_sample_info.tsv' % 'edited_'
sample_info = pd.read_csv(filepath, sep = '\t', na_filter = False)
sample_info = sample_info.set_index('entity:sample_id')
sample_info.index.name = 'sample_id' ## check: maybe delete this
sample_info = sample_info.reset_index()
sample_info = sample_info.set_index('sample_id')

print(sample_info.shape)
sample_info.head()

In [ ]:
# upload the TSVs
print("uploading new samples from tsv...")
refwm.upload_samples(sample_info)
# if not "NA" in refwm.get_samples().index.tolist():
#     refwm.upload_samples(pd.DataFrame({'sample_id':['NA'], 'participant_id':['NA']}).set_index('sample_id'))

## Uploading sample_set tsv to Terra from local file

In [ ]:
sampleset_members = pd.read_csv('../workspace_safety/DepMap_WES_CN_hg38/v4/sample_set/sample_set_membership.tsv', sep = "\t",na_filter= False)
sampleset_members = sampleset_members.rename(columns={"membership:sample_set_id":'sample_set_id'})
sampleset_members = sampleset_members.set_index('sample_set_id')

sampleset_entity = pd.read_csv('../workspace_safety/DepMap_WES_CN_hg38/v4/sample_set/sample_set_entity.tsv', sep = "\t", na_filter= False)
sampleset_entity = sampleset_entity.set_index('entity:sample_set_id')

In [ ]:
print("uploading sample_sets from tsvs...")
refname_work = refnamespace + '/' + refworkspace
# upload sample set entity
refwm.upload_entities('sample_set', sampleset_entity)

# upload sample set membership
grouped = sampleset_members.groupby('sample_set_id')
for name, group in grouped:
    try:
        terra.addToSampleSet(refname_work, samplesetid = name, samples = list(group['sample']))
    except KeyError:
        refwm.update_sample_set(name, list(group['sample']))